In [1]:
import pandas as pd

# 데이터 시각화
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
# 한글 폰트 문제 해결 
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

In [2]:
from tensorflow import keras

In [3]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [4]:
# train과 valid 분리
from sklearn.model_selection import train_test_split

In [8]:
# test 데이터 정규화
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

---
### Layer를 추가하는 방법

> 컬럼이 많을 때, 은닉층의 수는 입력층보다는 적고 출력층보다는 많게!

### 1

In [11]:
# Layer 만들기
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)) # 은닉
dense2 = keras.layers.Dense(10, activation='softmax') # 출력

In [12]:
# 전체 모델에 층 추가하기
model = keras.Sequential([dense1, dense2])

2022-07-20 14:36:00.141259: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# Summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


> Param # : 가중치 개수

### 2

In [15]:
model = keras.Sequential(
    [
        keras.layers.Dense(100, activation='sigmoid', input_shape=(784, ), name='hidden'),
        keras.layers.Dense(10, activation='softmax', name='output')
    ], name='패션 Mnist 모델'
)

In [16]:
model.summary()

Model: "패션 Mnist 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


### 3

> 주로 쓰는 방법

In [25]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [26]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 100)               78500     
                                                                 
 dense_11 (Dense)            (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
### 모델 훈련

In [27]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5653 - accuracy: 0.8074
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4088 - accuracy: 0.8538
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3739 - accuracy: 0.8648
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3514 - accuracy: 0.8730
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3350 - accuracy: 0.8798


In [28]:
import numpy as np

In [30]:
test_input.shape

(10000, 28, 28)

In [31]:
score = model.evaluate(test_input.reshape(-1, 784), np.array(test_target))
print('loss :', score[0])
print('accuracy :', score[1])

313/313 [==============================] - 1s 1ms/step - loss: 0.5102 - accuracy: 0.8276
loss : 0.5102137923240662
accuracy : 0.8276000022888184


![](../data/relu.png)

- Sigmoid 함수는 숫자가 무한대로 커지게되면 데이터의 차이가 거의 없다는 단점이 있다.
- 이를 해결한 것이 Relu함수인데, 인공신경망에 기여한 바가 크며, 음수는 모두 0으로 양수는 항상 최대값을 출력하는 함수이다.

In [32]:
# 입력층을 단순화
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28))) # 2차원으로 입력하면 알아서 펼쳐준다
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [33]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_12 (Dense)            (None, 100)               78500     
                                                                 
 dense_13 (Dense)            (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
### Optimizer(최적화 알고리즘)
- 훈련시에 Optimizer를 사용하여 최적화 알고리즘을 구성한다.
- 보통 사용하는 것이 확률적 경사 하강법 optimizer이다.

### 기본 경사 하강법
- SGD > 모멘텀 > 네스트로우 모멘텀
### 적응적 학습률 Optimizer
- 모멘텀 > Adam > RMSprop > Adagrad
- learning rate 0.001

### 확률적 경사 하강법 적용방법 1 : SGD

In [ ]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

### 확률적 경사 하강법 적용방법 2 : SGD

In [ ]:
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

### 확률적 경사 하강법 적용방법 3 : SGD
- 추가로 학습률 변경, SGD의 learning_rate의 기본값은 0.01

In [ ]:
sgd = keras.optimizers.SGD(learning_rate=0.1) # 기본값보다 10배정도 빠르다
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

### 확률적 경사 하강법 적용방법 4 : SGD

In [ ]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True) # momentum 0.9 : 하강하는 폭, nesterov True : 
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

### 확률적 경사 하강법 적용방법 5 : Adagrad

In [ ]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

### 확률적 경사 하강법 적용방법 6 : RMSprop

In [ ]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')